In [7]:
!pip install tabulate


This script monitors Bitcoin’s price by combining historical 24-hour data and real-time updates from Kraken’s public API. It first retrieves historical BTC/USD prices using the Kraken OHLC (Open-High-Low-Close) endpoint at 15-minute intervals, processes it into a pandas DataFrame, and plots it using Plotly. Simultaneously, it initializes an empty DataFrame to store live price data, fetched every 5 seconds using another Kraken API endpoint that returns the latest ticker price. Each new real-time data point is appended to the DataFrame, printed with a timestamp, and the plot is dynamically updated to show live market movement alongside the historical trend.

The code also gracefully handles interruptions: if the user stops the script (via KeyboardInterrupt), it merges the historical and real-time data and prints a clean tabulated history using the tabulate package. Error handling is integrated to manage issues like API failures or unexpected JSON structures. Overall, the script offers a lightweight yet powerful way to visualize and track Bitcoin's price trends both retrospectively and in real time.

In [10]:
import time
import requests
import pandas as pd
from datetime import datetime, timedelta
from tabulate import tabulate
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def get_bitcoin_price():
    # Kraken API endpoint for BTC/USD
    url = "https://api.kraken.com/0/public/Ticker?pair=XBTUSD"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        # Kraken's API structure is different, so adjust data access accordingly
        if 'result' in data and 'XXBTZUSD' in data['result']:
            price = data['result']['XXBTZUSD']['c'][0]
            return price
        else:
            print(f"Error: 'price' key not found in response. Response content: {data}")
            return None
    else:
        print(f"Error: Request failed with status code {response.status_code}")
        return None

def get_historical_data(since):
    # Kraken API endpoint for OHLC data
    url = f"https://api.kraken.com/0/public/OHLC?pair=XBTUSD&since={since}&interval=15"  # 15-minute interval
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if 'result' in data and 'XXBTZUSD' in data['result']:
            historical_data = data['result']['XXBTZUSD']
            df = pd.DataFrame(historical_data, columns=['time', 'open', 'high', 'low', 'close', 'vwap', 'volume', 'count'])
            df['time'] = pd.to_datetime(df['time'], unit='s')  # Convert Unix timestamp to datetime
            df.set_index('time', inplace=True)
            return df
        else:
            print(f"Error: Historical data not found in response. Response content: {data}")
            return None
    else:
        print(f"Error: Request failed with status code {response.status_code}")
        return None

if __name__ == "__main__":
    # Get historical data for the past 24 hours
    since = int((datetime.now() - timedelta(hours=24)).timestamp())
    historical_df = get_historical_data(since)

    if historical_df is not None:
        # Create an empty DataFrame to store real-time data
        df = pd.DataFrame(columns=['Date and Time', 'Price'])

        # Create Plotly figure
        fig = make_subplots(rows=1, cols=1)
        fig.add_trace(go.Scatter(x=historical_df.index, y=historical_df['close'], mode='lines', name='Historical Price'), row=1, col=1)
        fig.add_trace(go.Scatter(x=[], y=[], mode='lines', name='Real-Time Price'), row=1, col=1)
        fig.update_layout(title='Bitcoin Price (Historical and Real-Time)', xaxis_title='Date and Time', yaxis_title='Price (USD)')

        try:
            while True:
                price = get_bitcoin_price()
                if price:
                    # Get current date and time
                    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

                    # Append new data to the DataFrame
                    df = pd.concat([df, pd.DataFrame({'Date and Time': [now], 'Price': [price]})], ignore_index=True)

                    print(f"Current Bitcoin Price: ${price} at {now}")

                    # Update Plotly figure
                    fig.data[1].x = pd.to_datetime(df['Date and Time'])  # Convert to datetime for plotting
                    fig.data[1].y = df['Price']
                    fig.show()

                time.sleep(5)  # fetch every 5 seconds
        except KeyboardInterrupt:
            print("\nStopped.")

            # Display the table using tabulate
            print("\nBitcoin Price History:")
            print(tabulate(pd.concat([historical_df[['close']].reset_index(), df.rename(columns={'Price': 'close'})], ignore_index=True),
                          headers='keys', tablefmt='psql'))
    else:
        print("Error: Failed to retrieve historical data.")

Current Bitcoin Price: $94269.40000 at 2025-04-26 20:35:48


Current Bitcoin Price: $94269.40000 at 2025-04-26 20:35:53


Current Bitcoin Price: $94269.40000 at 2025-04-26 20:35:59


Current Bitcoin Price: $94269.40000 at 2025-04-26 20:36:04


Current Bitcoin Price: $94269.40000 at 2025-04-26 20:36:09


...

Current Bitcoin Price: $94269.40000 at 2025-04-26 20:36:36


Current Bitcoin Price: $94262.00000 at 2025-04-26 20:36:41


Current Bitcoin Price: $94262.00000 at 2025-04-26 20:36:46


Current Bitcoin Price: $94262.00000 at 2025-04-26 20:36:51



Stopped.

Bitcoin Price History:
+-----+---------------------+---------+---------------------+
|     | time                |   close | Date and Time       |
|-----+---------------------+---------+---------------------|
|   0 | 2025-04-25 20:45:00 | 94973.9 | nan                 |
|   1 | 2025-04-25 21:00:00 | 95006.6 | nan                 |
|   2 | 2025-04-25 21:15:00 | 94813.4 | nan                 |
|   3 | 2025-04-25 21:30:00 | 94822.1 | nan                 |
|   4 | 2025-04-25 21:45:00 | 94685.5 | nan                 |
|   5 | 2025-04-25 22:00:00 | 94849.9 | nan                 |
|   6 | 2025-04-25 22:15:00 | 94933   | nan                 |
|   7 | 2025-04-25 22:30:00 | 95015.5 | nan                 |
|   8 | 2025-04-25 22:45:00 | 94911.4 | nan                 |
|   9 | 2025-04-25 23:00:00 | 94951.1 | nan                 |
|  10 | 2025-04-25 23:15:00 | 94801.4 | nan                 |
|  11 | 2025-04-25 23:30:00 | 94817   | nan                 |
|  12 | 2025-04-25 23:45:00 | 94717.